## Question-4: Testing on Manually annotated dataset

In [3]:
from transformers import AutoModelForTokenClassification,AutoTokenizer

# Load the fine-tuned model
model = AutoModelForTokenClassification.from_pretrained('IndicBERT_model')

tokenizer=AutoTokenizer.from_pretrained('IndicBERT_model')

In [4]:
# Get the number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_parameters = count_parameters(model)
print("Number of trainable parameters:", num_parameters)

Number of trainable parameters: 32858375


In [5]:
import torch
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [104]:
# Open the text file in read mode
with open('25 Sentences.txt', 'r') as file:
  sentences=file.read().split('$')

In [7]:
label_id={ "LABEL_0":'O', "LABEL_1": "B-PER", "LABEL_2": "I-PER", "LABEL_3": "B-ORG", "LABEL_4":"I-ORG", "LABEL_5":"B-LOC", "LABEL_6":"I-LOC"}
for sentence in sentences:
  predicted_labels = get_predictions(sentence=sentence,
                                   tokenizer=tokenizer,
                                   model=model
                                   )

  words = sentence.split(' ')
  # Ensure the length of predicted labels matches the number of words
  if len(predicted_labels) > len(words):
    predicted_labels = predicted_labels[:len(words)]

  for i in range(len(words)):
    print( sentence.split(' ')[index] + '\t' + label_id[predicted_labels[index]])

నా	O
ప్రయాణం	O
మొత్తం	O
పదిరోజులే	O
కాబట్టి	O
నేను	O
మూడు	O
నాలుగు	O
ప్రదేశాల	O
కన్న	O
ఎన్నుకోలేను	O
,	O
ఒక్కో	O
చోటా	O
రెండు	O
మూడు	O
రోజుల	O
కన్న	O
ఎక్కువ	O
సమయం	O
ఉండలేను	O
.	O
ప్రజల	O
మనోభావాలు	O
,	O
ఎమోషన్స్	O
కి	O
అంతగా	O
ప్రాముఖ్యత	O
ఇవ్వకుండా	O
తానుఅనుకున్న	O
పనులు	O
చక్కబెట్టేవారు	O
.	O
మద్యం	O
లేదా	O
సాధ్యం	O
కిరోసిన్	O
ఉపయోగించడం	O
ద్వారా	O
మరింత	O
సమర్ధవంతంగా	O
జరిమానా	B-LOC
నష్టం	O
తొలగించడానికి	O
.	O
మరో	O
వివాహం	O
చేసుకునేందుకు	O
వీలుగా	O
భార్య	O
పాయల్‌	O
నుంచి	O
విడాకులు	O
ఇప్పించాలి	O
'	O
అని	O
ఒమర్	B-PER
పిటిషన్‌లో	O
విన్నవించారు	O
.	O
అన్ని	O
ఇతర	O
ఈస్ట్	O
వంటి	O
,	O
ఈ	O
జాతులు	O
చాలా	O
విటమిన్లు	O
మరియు	O
ట్రేస్	O
ఎలిమెంట్స్	O
చాలా	O
గొప్ప	O
ఉంది	O
,	O
ప్రోటీన్	O
చాలా	O
కలిగి	O
మరియు	O
మొత్తం	O
చాలా	O
విలువైన	O
కూర్పు	O
ఉంది	O
.	O
జి.ఎస్‌.టి.	B-ORG
కౌన్సిల్‌	O
సమావేశం	I-ORG
తరువాత	O
జైట్లీ	O
ప్రకటించిన	O
ప్యాకేజీలో	O
చిన్న	O
,	O
మధ్యతరహా	O
పరిశ్రమాధిపతులకు	O
,	O
చేయూతనందించే	O
పథకంగా	O
ఉందని	O
ప్రశంసించారు	O
.	O
వాస్తవానికి	O
ఈ	O
పారిశ్రామిక	O
వర్గం	O
యొక్క	O
అవస్థల

In [ ]:
# Open a text file in write mode
with open('predicted_labels_bert.txt', 'w') as file:
  for sentence in sentences:
    predicted_labels = get_predictions(sentence=sentence,
                                      tokenizer=tokenizer,
                                      model=model)
    #print(len(sentence.split(' ')),len(predicted_labels))

    for index in range(len(sentence.split(' '))):
      if (sentence.split(' ')[index]!=" "):
        file.write(sentence.split(' ')[index] + '\t' +  label_id[predicted_labels[index]] + '\n')

In [122]:
def read_and_tokenize_file(file_path, tokenizer):
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:
                line_data = eval(line)  # Convert the string representation of the dictionary to a dictionary
                tokens = line_data["tokens"]
                ner_tags = line_data["ner_tags"]
                line_dict = {"tokens": tokens, "ner_tags": ner_tags}
                data.append(line_dict)
    return data


# Specify the path to the text file
file_path = "colab_maual.txt"

# Read and tokenize the file
tokenized_data = read_and_tokenize_file(file_path, tokenizer)
ground_truth_tokens=[]
ground_truth_tokenized_labels=[]
# Print the tokenized data
c=0
for line_data in tokenized_data:
  # c+=1
  # if(c==4):
  #   for i in range(15):
  #     print(line_data["tokens"][i],line_data["ner_tags"][i])
  if("," in line_data["ner_tags"]):
    for i in range(len(line_data["ner_tags"])):
      print(line_data["tokens"][i],line_data["ner_tags"][i])
      if(line_data["ner_tags"][i]=="OO"): print("culprit ",i)
  ground_truth_tokens.extend(line_data["tokens"])
  ground_truth_tokenized_labels.extend(line_data["ner_tags"])
  #print(len(line_data["tokens"]),len(line_data["ner_tags"]))

In [107]:
label_id={ "LABEL_0":'O', "LABEL_1": "B-PER", "LABEL_2": "I-PER", "LABEL_3": "B-ORG", "LABEL_4":"I-ORG", "LABEL_5":"B-LOC", "LABEL_6":"I-LOC"}
import torch
predicted_labels_global=[]
predicted_tokens=[]
for sentence in sentences:
  predicted_labels = get_predictions(sentence=sentence,
                                    tokenizer=tokenizer,
                                    model=model)

  words = sentence.split(' ')
  # Ensure the length of predicted labels matches the number of words
  if len(predicted_labels) > len(words):
    predicted_labels = predicted_labels[:len(words)]
  print(len(words),len(predicted_labels))
  for i in range(len(words)):
    #print(words[i], predicted_labels[i])
    predicted_tokens.append(words[i])
    predicted_labels_global.append( label_id[predicted_labels[i]])

22 22
12 12
12 12
15 15
26 26
144 144
13 13
13 13
24 24
16 16
11 11
24 24
43 43
27 27
16 16
11 11
14 14
13 13
25 25
19 19
16 16
32 32
15 15
14 14
14 14



In [108]:
print(predicted_labels_global)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [123]:
print(len(predicted_labels_global),len(ground_truth_tokenized_labels))

591 591


In [124]:
print(len(ground_truth_tokens),len(predicted_tokens))

591 591


In [125]:
from sklearn.metrics import precision_score, recall_score, f1_score,classification_report
def calculate_metrics(predicted_labels, ground_truth_labels):
    precision = precision_score(ground_truth_labels, predicted_labels, average='macro')
    recall = recall_score(ground_truth_labels, predicted_labels, average='macro')
    f1_sco = f1_score(ground_truth_labels, predicted_labels, average='macro')
    #report=classification_report(ground_truth_labels, predicted_labels)
    return precision, recall, f1_sco

In [126]:
p,r,f1=calculate_metrics(predicted_labels_global,ground_truth_tokenized_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [127]:
print("Precision: ",p,"\n","Recall: ",r,"\n","F1-Score: ",f1)

Precision:  0.3923867974500886 
 Recall:  0.3443784924688974 
 F1-Score:  0.3629307783719548


In [128]:
report=classification_report(ground_truth_tokenized_labels,predicted_labels_global)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.60      0.50      0.55         6
      B-MISC       0.00      0.00      0.00        27
       B-ORG       0.67      0.40      0.50         5
       B-PER       0.73      0.58      0.65        19
       I-LOC       0.00      0.00      0.00         1
      I-MISC       0.00      0.00      0.00        15
       I-ORG       0.00      0.00      0.00         4
       I-PER       0.64      0.64      0.64        11
           O       0.90      0.98      0.94       503

    accuracy                           0.88       591
   macro avg       0.39      0.34      0.36       591
weighted avg       0.81      0.88      0.84       591



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
